# Stable Mariages

For this workshop we'll implement a version of the **Nobel Prize-winning algorithm** for the stable marriage problem. It's called the [Gale-Shapley](https://en.wikipedia.org/wiki/Gale%E2%80%93Shapley_algorithm) algorithm AKA the "Deferred Acceptance" algorithm.

The Gale Shapley algorithm solves what's called the **Stable Marriage** Problem. Here is the problem framed as marrying men and women: 

> Given $n$ men and $n$ women, where each person has ranked all members of the opposite sex in order of preference, marry the men and women together such that there are no two people of opposite sex who would both rather have each other than their current partners. When there are no such pairs of people, the set of marriages is deemed stable.

This little model however is generally applied to more than marrying men and women. For example, it's the algorithm used to match newly graduated doctors with hospitals, and a variation on it matches kidney patients with organ donors in our hospital system.

The algorithmic question is, given lists of preferences as input, can we find a stable marriage? Can we even guarantee a stable marriage will exist for any set of preferences? The answer to both questions is yes, and it uses an algorithm called deferred acceptance.

Here is an informal description of the algorithm. It goes in rounds. In each round, each man “proposes” to the highest-preferred woman that has not yet rejected him. On the other side, each woman holds a reference to a man at all times. If a woman gets new proposals in a round, she immediately rejects every proposer except her most preferred, but does not accept that proposal. She “defers” the acceptance of the proposal until the very end.

![](Gale-Shapley.gif)

# Your Task

Build a solution `gale_shapley(men, women) -> dict` Where the input is a list of suitors and a list of Suiteds with every one in these lists holding their lists of preferences. The output is a dictionary `suitor -> suited`

Here is a possible way to design a solution:

### Man Class

The `Man(id, preference_list)` class holds the following attributes:

- Its own ID (an `int`)

- A list or array of IDs, which are ordered. So `preference_list[0]` is prefered to `preference_list[1]` and so on

- A method `Suitor.preference()` which points to its current possible reference. It should start by pointing at `preference_list[0]` and every time the Suitor gets rejected in the algorithm, this should point to the next preference.

### Woman Class

The `Woman(id, preference_list)` class holds the following attributes:

- Its own ID (an `int`)

- A list or array of IDs, which are ordered. So `preference_list[0]` is prefered to `preference_list[1]` and so on

- A set of current suitors

- A method `Suited.reject()` which returns all current suitors except the most preferred one

### The Stable Mariage Algorithm

Takes in a list of men and women, loops over suitors trying to find a match until there aren't any unassigned suitors left. Here is the pseudocode for the algorithm:

```
algorithm stable_matching
    Initialize all m ∈ M and w ∈ W to free
    while ∃ free man m who still has a woman w to propose to do
        w := first woman on m's list to whom m has not yet proposed
        if w is free then
            (m, w) become engaged
        else some pair (m', w) already exists
            if w prefers m to m' then
                m' becomes free
                (m, w) become engaged 
            else
                (m', w) remain engaged
            end if
        end if
    repeat
```


Here is some example code of a solution working:

```
men = [
    Man(id=0, preference_list=[0,1,2,3]),
    Man(id=1, preference_list=[2,3,0,1]),
    Man(id=2, preference_list=[1,0,3,2]),
    Man(id=3, preference_list=[3,2,1,0]),
]

women = [
    Woman(id=0, preference_list=[0,1,2,3]),
    Woman(id=1, preference_list=[2,3,0,1]),
    Woman(id=2, preference_list=[1,0,3,2]),
    Woman(id=3, preference_list=[3,2,1,0]),
]

stable_marriage(men, women)
```

output:

```
{0: 0, 
 2: 1, 
 1: 2, 
 3: 3}
```

In [1]:
"""

BRUNO UGOLINI

"""


class Man():
    """
    Defines a class and associated
    methods for a man in the stable
    marriage problem.
    """
    
    def __init__(self, idx='', preference_list=[]):
        #initialize index and preference list
        self.id = idx
        self.prflst = preference_list
        
    def preference(self):
        try:
            # set index for preference
            # and increment it
            nxt = self.prflst[self.next]
            self.next += 1
        except AttributeError:
            # do likewise but for the
            # first call
            nxt = self.prflst[0]
            self.next = 1
        except IndexError:
            # if we hit this message
            # something went wrong in main.
            return print(f"Man ID[{self.id}] has exhausted preferences!")
        return nxt
    
    def __repr__(self):
        return str([self.id, self.prflst])
    
    def __getitem__(self, key):
        return [self.id, self.prflst][key]

class Woman():
    """
    Defines a class and associated
    methods for a woman in the stable
    marriage problem.
    """
    
    def __init__(self, idx='', preference_list=[]):
        # initialize index and preference list
        self.id = idx
        self.prflst = preference_list
        
    def reject(self, suitor):
        try:
            if self.prflst.index(suitor) < self.best:
                # if this suitor is more to the left
                # by index in the preference list
                # swap him out as the best choice.
                rejected = self.prflst[self.best]
                self.best = self.prflst.index(suitor)
                # return the old one as rejected
                return rejected
            else:
                # return the suitor as rejected otherwise
                return suitor
        except AttributeError:
            # take care of first call settings
            self.best = self.prflst.index(suitor)
            return -1

    def __getitem__(self, key):
        return [self.id, self.prflst][key]
        
    def __repr__(self):
        return str([self.id, self.prflst])

def gale_shapely(men, women):
    """
    Function to calculate the stable
    marriage solution given a set of
    men(id's, preference lists) and 
    women(same attributes). Output is
    a dictionary with the marriages
    defined.
    """
    
    # initialize the solution dictionary.
    res = {}
    for man in men:
        res[man[0]] = -1
    
    # while men are still available ...
    while -1 in res.values():
        
        for k, v in res.items():
            
            # if a mate was not found
            if v < 0:
                
                # get next bride
                nxt_bride = men[k].preference()
                # check for rejection
                rejected = women[nxt_bride].reject(k)
                
                if rejected < 0:
                    # if first caller
                    res[k] = nxt_bride
                elif rejected != k:
                    # set new bride if success
                    res[k] = nxt_bride
                    # reset old bridegroom to
                    # available status
                    res[rejected] = -1
                    
    return res
        
        
men = [
    Man(idx=0, preference_list=[0,1,2,3]),
    Man(idx=1, preference_list=[2,3,0,1]),
    Man(idx=2, preference_list=[1,0,3,2]),
    Man(idx=3, preference_list=[3,2,1,0]),
]
print("Men :")
for man in men:
    print(man)
women = [
    Woman(idx=0, preference_list=[0,1,2,3]),
    Woman(idx=1, preference_list=[2,3,0,1]),
    Woman(idx=2, preference_list=[1,0,3,2]),
    Woman(idx=3, preference_list=[3,2,1,0]),
]
print("\nWomen :")
for woman in women:
    print(woman)
print("\n")
results = gale_shapely(men,women)
for m, w in results.items():
    print(f"Man [{m}] marries Woman [{w}].") 

Men :
[0, [0, 1, 2, 3]]
[1, [2, 3, 0, 1]]
[2, [1, 0, 3, 2]]
[3, [3, 2, 1, 0]]

Women :
[0, [0, 1, 2, 3]]
[1, [2, 3, 0, 1]]
[2, [1, 0, 3, 2]]
[3, [3, 2, 1, 0]]


Man [0] marries Woman [0].
Man [1] marries Woman [2].
Man [2] marries Woman [1].
Man [3] marries Woman [3].


In [2]:
"""
The following tests my own
random number selections of
man/woman preferences.
"""

men = [
    Man(idx=0, preference_list=[1,2,3,4,0,5]),
    Man(idx=1, preference_list=[2,1,5,0,3,4]),
    Man(idx=2, preference_list=[5,0,1,4,2,3]),
    Man(idx=3, preference_list=[4,2,1,3,5,0]),
    Man(idx=4, preference_list=[1,5,3,0,2,4]),
    Man(idx=5, preference_list=[3,0,2,4,5,1]),
]
print("Men :")
for man in men:
    print(man)
women = [
    Woman(idx=0, preference_list=[2,5,3,4,1,0]),
    Woman(idx=1, preference_list=[2,4,5,0,3,1]),
    Woman(idx=2, preference_list=[1,2,4,5,3,0]),
    Woman(idx=3, preference_list=[3,2,4,5,0,1]),
    Woman(idx=4, preference_list=[1,0,3,5,2,4]),
    Woman(idx=5, preference_list=[2,3,0,4,5,1]),
]
print("\nWomen :")
for woman in women:
    print(woman)
print("\n")
results = gale_shapely(men,women)
for m, w in results.items():
    print(f"Man [{m}] marries Woman [{w}].") 

Men :
[0, [1, 2, 3, 4, 0, 5]]
[1, [2, 1, 5, 0, 3, 4]]
[2, [5, 0, 1, 4, 2, 3]]
[3, [4, 2, 1, 3, 5, 0]]
[4, [1, 5, 3, 0, 2, 4]]
[5, [3, 0, 2, 4, 5, 1]]

Women :
[0, [2, 5, 3, 4, 1, 0]]
[1, [2, 4, 5, 0, 3, 1]]
[2, [1, 2, 4, 5, 3, 0]]
[3, [3, 2, 4, 5, 0, 1]]
[4, [1, 0, 3, 5, 2, 4]]
[5, [2, 3, 0, 4, 5, 1]]


Man [0] marries Woman [4].
Man [1] marries Woman [2].
Man [2] marries Woman [5].
Man [3] marries Woman [3].
Man [4] marries Woman [1].
Man [5] marries Woman [0].


In [3]:
"""
First additional example
provided by Matt.
"""

men = [
    Man(idx=0, preference_list=[0,4,1,5,2,3]),
    Man(idx=1, preference_list=[4,2,5,3,0,1]),
    Man(idx=2, preference_list=[4,1,0,3,2,5]),
    Man(idx=3, preference_list=[4,3,2,1,0,5]),
    Man(idx=4, preference_list=[4,2,3,1,0,5]),
    Man(idx=5, preference_list=[0,1,4,2,3,5]),
]
print("Men :")
for man in men:
    print(man)
women = [
    Woman(idx=0, preference_list=[0,1,4,2,3,5]),
    Woman(idx=1, preference_list=[4,2,3,0,1,5]),
    Woman(idx=2, preference_list=[5,1,0,3,4,2]),
    Woman(idx=3, preference_list=[5,4,3,2,1,0]),
    Woman(idx=4, preference_list=[5,3,4,2,1,0]),
    Woman(idx=5, preference_list=[0,1,4,2,3,5]),
]
print("\nWomen :")
for woman in women:
    print(woman)
print("\n")
results = gale_shapely(men,women)
for m, w in results.items():
    print(f"Man [{m}] marries Woman [{w}].") 

Men :
[0, [0, 4, 1, 5, 2, 3]]
[1, [4, 2, 5, 3, 0, 1]]
[2, [4, 1, 0, 3, 2, 5]]
[3, [4, 3, 2, 1, 0, 5]]
[4, [4, 2, 3, 1, 0, 5]]
[5, [0, 1, 4, 2, 3, 5]]

Women :
[0, [0, 1, 4, 2, 3, 5]]
[1, [4, 2, 3, 0, 1, 5]]
[2, [5, 1, 0, 3, 4, 2]]
[3, [5, 4, 3, 2, 1, 0]]
[4, [5, 3, 4, 2, 1, 0]]
[5, [0, 1, 4, 2, 3, 5]]


Man [0] marries Woman [0].
Man [1] marries Woman [2].
Man [2] marries Woman [1].
Man [3] marries Woman [5].
Man [4] marries Woman [3].
Man [5] marries Woman [4].


In [4]:
"""
Second additional example
provided by Matt.
"""

men = [
    Man(idx=0, preference_list=[0,4,1,2,3]),
    Man(idx=1, preference_list=[4,2,3,0,1]),
    Man(idx=2, preference_list=[4,1,0,3,2]),
    Man(idx=3, preference_list=[4,3,2,1,0]),
    Man(idx=4, preference_list=[4,2,3,1,0]),
]
print("Men :")
for man in men:
    print(man)
women = [
    Woman(idx=0, preference_list=[0,1,4,2,3]),
    Woman(idx=1, preference_list=[4,2,3,0,1]),
    Woman(idx=2, preference_list=[1,0,3,4,2]),
    Woman(idx=3, preference_list=[4,3,2,1,0]),
    Woman(idx=4, preference_list=[3,4,2,1,0]),
]
print("\nWomen :")
for woman in women:
    print(woman)
print("\n")
results = gale_shapely(men,women)
for m, w in results.items():
    print(f"Man [{m}] marries Woman [{w}].") 

Men :
[0, [0, 4, 1, 2, 3]]
[1, [4, 2, 3, 0, 1]]
[2, [4, 1, 0, 3, 2]]
[3, [4, 3, 2, 1, 0]]
[4, [4, 2, 3, 1, 0]]

Women :
[0, [0, 1, 4, 2, 3]]
[1, [4, 2, 3, 0, 1]]
[2, [1, 0, 3, 4, 2]]
[3, [4, 3, 2, 1, 0]]
[4, [3, 4, 2, 1, 0]]


Man [0] marries Woman [0].
Man [1] marries Woman [2].
Man [2] marries Woman [1].
Man [3] marries Woman [4].
Man [4] marries Woman [3].
